In [214]:
import pandas as pd
import sys
print sys.version
import os
import glob
from IPython.display import display, HTML

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [27]:
honda=pd.read_csv("../Data/OPP-115/annotations/1050_honda.com.csv", header=None)
honda.columns=["ID", "Type", "number", "number2", "section", "category", "annotation", "date", "url"]
honda.head(3)

,ID,Type,number,number2,section,category,annotation,date,url
0,20842,test_category_labeling_highlight_upitt,95,3836,0,Other,"{""Other Type"": {""selectedText"": ""STATEMENT OF ...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15695,test_category_labeling_highlight_upitt,103,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx
2,20843,test_category_labeling_highlight_upitt,95,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [21]:
hondaAnnot=honda.loc[:,["section", "category", "annotation"]]

In [22]:
group=list(hondaAnnot.groupby(["category"]))

In [55]:
print group[1][1]


     section       category                                         annotation
66        14  Data Security  {"Security Measure": {"selectedText": "America...
67        14  Data Security  {"Security Measure": {"selectedText": "The fol...
68        14  Data Security  {"Security Measure": {"selectedText": "In addi...
69        14  Data Security  {"Security Measure": {"selectedText": "However...
70        14  Data Security  {"Security Measure": {"selectedText": "This in...
71        14  Data Security  {"Security Measure": {"selectedText": "In addi...
201       62  Data Security  {"Security Measure": {"selectedText": "America...
203       63  Data Security  {"Security Measure": {"selectedText": "How do ...
204       63  Data Security  {"Security Measure": {"selectedText": "We main...
264       14  Data Security  {"Security Measure": {"selectedText": "America...
265       14  Data Security  {"Security Measure": {"selectedText": "This in...
266       14  Data Security  {"Security Measure": {"

In [33]:
annolist=group[3][1]["annotation"].values
print group[3][0]
eval(group[3][1]["annotation"].iloc[0])["Purpose"]

First Party Collection/Use


{'endIndexInSegment': -1,
 'selectedText': 'Not selected',
 'startIndexInSegment': -1,
 'value': 'Unspecified'}

In [ ]:
for key in eval(annolist[0]):
  print key
  print eval(annolist[0])[key]

In [147]:

## in some texts, not all annotation have the same attributes
abita=pd.read_csv("../Data/OPP-115/annotations/571_abita.com.csv", header=None)
abita.columns=["ID", "Type", "number", "number2", "section", "category", "annotation", "date", "url"]

abitaGroup=list(abita[[ "category", "annotation", "section"]].groupby(["category"]))
"""for index in abitaGroup[1][1].index:
  annoRecord=eval(abitaGroup[1][1].ix[index, "annotation"])
  print [len(annoRecord[topic].keys()) for topic in annoRecord]
"""

print eval(abitaGroup[1][1].iloc[0,1])["Collection Mode"].keys()  # Collection Mode in the first annotation in the category
print eval(abitaGroup[1][1].iloc[4,1])["Collection Mode"].keys()  # Collection Mode in the 5th annotation in the category

['endIndexInSegment', 'startIndexInSegment', 'value']
['endIndexInSegment', 'startIndexInSegment', 'selectedText', 'value']


In [169]:
import json
abitaFP=abitaGroup[1][1]
abitaFPanno=abitaGroup[1][1]["annotation"]
abitaFPannoEg=eval(abitaFPanno.iloc[0])
abitaFPannoEg2=eval(abitaFPanno.iloc[5])
#pd.DataFrame.from_dict([eval(anno) for anno in abitaFPanno], orient="index")
pds1=pd.Series(abitaFPannoEg["Action First-Party"])
pds2=pd.Series(abitaFPannoEg2["Action First-Party"])
pd.DataFrame([pds2, pds1])

,endIndexInSegment,selectedText,startIndexInSegment,value
0,34,We may collect certain information,0,Unspecified
1,531,personal information we collect,500,Unspecified


## Functions for packing annotations

In [210]:
## take two cateogries, put the values of the same key together in a list
def joinDict(origin, newdict):
  for key in origin:
    try:
      origin[key].extend([newdict[key]])
    except KeyError:
      origin[key].append("NaN")
  return origin

## extract all the subcategory (topics) in a category, and gather the values of each topic into a 
## dataframe along with section value
def packCategory(categoryFrame):
  annocollect={}
  annoFrames={}
  for rownumber in range(categoryFrame.shape[0]):
    anno=eval(categoryFrame["annotation"].iloc[rownumber])
    sectionValue=categoryFrame["section"].iloc[rownumber]
    for topic in anno:
      annocollect.setdefault(topic, [])
      topicAnno=anno[topic]
      topicAnno["section"]=sectionValue
      annocollect[topic].append(pd.Series(topicAnno))

    for topic in anno:
      frame=pd.DataFrame(annocollect[topic])
      #frame=frame[['selectedText','section','startIndexInSegment','endIndexInSegment','value']]
      annoFrames[topic]=frame
    
  return annoFrames

## pack each category in a list
def packPolicy(policyDataFrame):
  totalresult={}
  filtered=policyDataFrame.loc[:,[ "category", "annotation", "section"]]
  grouped=list(filtered.groupby(["category"]))
  for group in grouped:
    category=group[0]
    categoryData=group[1]
    totalresult[category]=packCategory(categoryData)
  return totalresult

def parseAllAnnotations(fileList):
  fulloutput={}
  for filepath in fileList:
    website=os.path.splitext(os.path.basename(filepath))[0].split("_")[1]
    annoData=pd.read_csv(filepath, header=None, names=["number", "Type", "ID", "number2", "section", "category", "annotation", "date", "url"])
    parsedAnno=packPolicy(annoData)
    fulloutput[website]=parsedAnno
  return fulloutput


In [201]:
eval(abitaGroup[1][1]["annotation"].iloc[0])['Collection Mode'].keys()

['endIndexInSegment', 'startIndexInSegment', 'value']

In [208]:
test=packCategory(abitaGroup[1][1])


In [205]:
result=packPolicy(honda)
result["First Party Collection/Use"]['Collection Mode'].keys()

Index([u'endIndexInSegment', u'section', u'selectedText',
       u'startIndexInSegment', u'value'],
      dtype='object')

In [211]:
%%time
## pack all original annotations files
originalAnnoFiles=glob.glob("../Data/OPP-115/annotations/*.csv")
packedAnnoOriginal=parseAllAnnotations(originalAnnoFiles)

CPU times: user 7min 40s, sys: 407 ms, total: 7min 40s
Wall time: 7min 40s


In [216]:
print packedAnnoOriginal["foxsports.com"].keys()
print "\n"
print packedAnnoOriginal["foxsports.com"]["Data Retention"].keys()
print "\n"
print packedAnnoOriginal["foxsports.com"]["Data Retention"]["Retention Period"].columns
print "\n"
display(packedAnnoOriginal["foxsports.com"]["Data Retention"]["Retention Period"])

['Data Retention', 'First Party Collection/Use', 'User Choice/Control', 'Policy Change', 'Do Not Track', 'International and Specific Audiences', 'User Access, Edit and Deletion', 'Other', 'Data Security', 'Third Party Sharing/Collection']


['Personal Information Type', 'Retention Period', 'Retention Purpose']


Index([u'endIndexInSegment', u'section', u'selectedText',
       u'startIndexInSegment', u'value'],
      dtype='object')




,endIndexInSegment,section,selectedText,startIndexInSegment,value
0,234,35,We will retain your information for the period...,47,Limited
1,159,35,We will retain your information for the period...,47,Other
2,773,36,Remember that even after you cancel your accou...,0,Limited
3,402,36,We may also retain backup information related ...,206,Unspecified
4,234,35,or allowed by applicable law.,205,Unspecified
5,192,35,unless a longer retention period,160,Unspecified
6,314,36,for some time after cancelation,283,Unspecified
7,314,36,for some time after cancelation,283,Unspecified


In [223]:
packedAnnoOriginal["foxsports.com"]["Data Retention"]["Retention Period"].

AttributeError: 'DataFrame' object has no attribute 'to_list'

In [217]:
import pickle
with open("Data/parsed-annotation-original.pk", 'wb') as f:
  pickle.dump(packedAnnoOriginal, f)

In [24]:
import json
for key in packedAnnoOriginal:
  with open(os.path.join("../Data/parsed-annotation-original",key)+".json", "w") as f:
    json.dump(packedAnnoOriginal[key], f)

## Organize consolidated annotations

In [218]:
#/Users/NYao/Documents/MIDS/210/Data/OPP-115/consolidation/
hondaConso=pd.read_csv("../Data/OPP-115/consolidation/threshold-1.0-overlap-similarity/1050_honda.com.csv")
hondaConso.columns=["Number", "Type", "annotator", "number2", "number3", "category", "annotation", "date", "url"]
hondaConso.head(2)

,Number,Type,annotator,number2,number3,category,annotation,date,url
0,C549,test_category_labeling_highlight_fordham_aaaaa,121,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15696,test_category_labeling_highlight_upitt,103,3836,2,Other,"{""Other Type"": {""selectedText"": ""This statemen...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [219]:
hondaConsoPack=packPolicy(hondaConso)

In [220]:
consoAnno10=glob.glob("../Data/OPP-115/consolidation/threshold-1.0-overlap-similarity/*.csv")
packedAnnoConso10=parseAllAnnotations(consoAnno10)

In [30]:
print len(consoAnno10)

115


In [224]:
with open("Data/parsed-annotation-1.0.pk", 'wb') as f:
  pickle.dump(packedAnnoOriginal, f)

In [34]:
for key in packedAnnoConso10:
  with open(os.path.join("Data/opp115-parsed-annotation-1.0",key)+".json", "w") as f:
    json.dump(packedAnnoConso10[key], f)

## Consolidated Annotation with 0.5 threshold


In [226]:
consoAnno05=glob.glob("../Data/OPP-115/consolidation/threshold-0.5-overlap-similarity/*.csv")
print(len(consoAnno05))
packedAnnoConso05=parseAllAnnotations(consoAnno05)

115


## Organize human-redable summaries

In [222]:
for i in range(10):
  print(i, flush=True)

SyntaxError: invalid syntax (<ipython-input-222-668a0046ee05>, line 2)

## Organize Privacy Policy Texts